In [1]:
import pandas as pd
import gc
import numpy as np
from sklearn.preprocessing import RobustScaler
import optuna
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam, RMSprop, Nadam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

study_name = "lstm_opt_2019"

df = pd.read_csv("../../datasets/sell-in.txt.gz", sep="\t")

productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
df = df[df["product_id"].isin(productos_ok["product_id"])]
df = df.groupby(by=["periodo","product_id"]).agg({"tn":"sum"}).reset_index()
df["periodo"] = pd.to_datetime(df["periodo"], format="%Y%m")
df_pivot = df.pivot(index="product_id", columns="periodo", values="tn").fillna(0)
print(f"Shape de la matriz: {df_pivot.shape}")  # (800 productos x 36 meses)
del df
gc.collect()

c:\Users\carre\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Shape de la matriz: (780, 36)


23

In [2]:
scalers = {}  # Guardamos los scalers para invertir luego
scaled_data = np.zeros_like(df_pivot.values)

for i, (producto, fila) in enumerate(df_pivot.iterrows()):
    scaler = RobustScaler()
    scaled_data[i] = scaler.fit_transform(fila.values.reshape(-1, 1)).flatten()
    scalers[producto] = scaler  # Almacenar scaler para este producto

In [3]:
X, y, periodos_target = [], [], []

for i_producto, producto in enumerate(df_pivot.index):
    serie = scaled_data[i_producto]
    for i in range(12, len(serie) - 2):  # Asegurar que t+2 exista

        y.append(serie[i+2])  # t+2 (i+1 es el offset desde el fin de la ventana)
        periodos_target.append(df_pivot.columns[i + 2])
        per = df_pivot.columns[i]  # el último mes dentro de la ventana
        mes = pd.to_datetime(per, format="%Y%m").month
        quarter = pd.to_datetime(per, format="%Y%m").quarter
        
        mes_scaled = (mes - 1) / 11  # Normalizado a [0, 1]
        quarter_scaled = (quarter - 1) / 3  # quarter ∈ [0, 3]
        
        X.append(
            np.concatenate([
            serie[i-12:i].reshape(-1, 1),  # Shape: (12, 1)
            np.array([[mes_scaled], [quarter_scaled]])  # Shape: (2, 1)
            ], axis=0)
        )  # Shape final: (14, 1)
        

X = np.array(X)  # Shape: (n_muestras, 14, 1)
y = np.array(y).reshape(-1, 1)  # Shape: (n_muestras, 1)
periodos = np.array(periodos_target)
# Ahora sí, aplicá las máscaras
train_mask = (periodos != pd.to_datetime("2019-02-01")) & (periodos <= pd.to_datetime("2019-12-31"))
test_mask = periodos == pd.to_datetime("2019-02-01")

X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y[train_mask], y[test_mask]

In [16]:

def build_model(trial):

    l1 = trial.suggest_float("l1", 1e-8, 1e-3, log=True)
    l2 = trial.suggest_float("l2", 1e-8, 1e-2, log=True)
    reg = regularizers.l1_l2(l1=l1, l2=l2)

    model = Sequential()
    model.add(LSTM(
        units=trial.suggest_categorical("lstm_units_1", [32, 64, 128]),
        activation=trial.suggest_categorical("activation_lstm", ["tanh", "relu"]),
        input_shape=(14, 1),
        return_sequences=True,
        recurrent_dropout=trial.suggest_float("recurrent_dropout_1", 0.0, 0.5),
        kernel_regularizer=reg
    ))
    model.add(Dropout(trial.suggest_float("dropout_rate", 0.1, 0.5)))

    model.add(LSTM(
        units=trial.suggest_categorical("lstm_units_2", [16, 32, 64]),
        activation=trial.suggest_categorical("activation_lstm_2", ["tanh", "relu"]),
        recurrent_dropout=trial.suggest_float("recurrent_dropout_2", 0.0, 0.5),
        kernel_regularizer=reg
    ))

    model.add(Dense(
        units=trial.suggest_categorical("dense_units", [8, 16, 32]),
        activation=trial.suggest_categorical("activation_dense", ["relu", "linear"]),
        kernel_regularizer=reg
    ))

    model.add(Dense(1))

    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "rmsprop"])
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

    if optimizer_name == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == "rmsprop":
        optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    else:
        optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

    model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
    return model


def objective(trial):
    model = build_model(trial)

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=200,
        batch_size=128,
        verbose=1,
        callbacks=[early_stopping]
    )

    val_loss = min(history.history["val_loss"])
    return val_loss

os.makedirs("optuna_storage", exist_ok=True)
DB_PATH = "optuna_storage/bbdd.db"
STUDY_NAME = study_name
storage_url = f"sqlite:///{DB_PATH}"

study = optuna.create_study(
    study_name=STUDY_NAME,
    storage=storage_url,
    direction="minimize",
    load_if_exists=True
)


# Ejecutar la búsqueda

study.optimize(objective, n_trials=50)

# Mostrar mejores parámetros
print("Mejores parámetros:")
print(study.best_params)


[I 2025-07-09 14:57:04,620] A new study created in RDB with name: lstm_opt_2019


Epoch 1/200
128/128 [==============================] - 39s 258ms/step - loss: 88.8228 - mae: 1.4810 - val_loss: 1.8982 - val_mae: 0.8767
Epoch 2/200
128/128 [==============================] - 32s 249ms/step - loss: 85.9731 - mae: 1.5582 - val_loss: 1.7581 - val_mae: 0.9309
Epoch 3/200
128/128 [==============================] - 32s 248ms/step - loss: 83.6576 - mae: 1.5699 - val_loss: 1.1849 - val_mae: 0.8159
Epoch 4/200
128/128 [==============================] - 32s 250ms/step - loss: 80.6840 - mae: 1.5896 - val_loss: 2.3772 - val_mae: 1.2126
Epoch 5/200
128/128 [==============================] - 32s 248ms/step - loss: 82.4047 - mae: 1.6361 - val_loss: 1.7329 - val_mae: 0.5591
Epoch 6/200
128/128 [==============================] - 32s 250ms/step - loss: 80.1088 - mae: 1.6416 - val_loss: 1.3236 - val_mae: 0.7417
Epoch 7/200
128/128 [==============================] - 32s 251ms/step - loss: 81.4699 - mae: 1.6862 - val_loss: 0.9958 - val_mae: 0.6189
Epoch 8/200
128/128 [====================

[I 2025-07-09 15:11:37,493] Trial 0 finished with value: 0.9222404956817627 and parameters: {'l1': 0.0005772229507253608, 'l2': 2.9119285595315917e-05, 'lstm_units_1': 32, 'activation_lstm': 'relu', 'recurrent_dropout_1': 0.12303749939045538, 'dropout_rate': 0.41345078620888676, 'lstm_units_2': 16, 'activation_lstm_2': 'tanh', 'recurrent_dropout_2': 0.06322661072271746, 'dense_units': 32, 'activation_dense': 'linear', 'optimizer': 'rmsprop', 'learning_rate': 0.0062212907616810914}. Best is trial 0 with value: 0.9222404956817627.


Epoch 1/200
128/128 [==============================] - 39s 262ms/step - loss: 94.0245 - mae: 1.2729 - val_loss: 0.7210 - val_mae: 0.4294
Epoch 2/200
128/128 [==============================] - 33s 261ms/step - loss: 93.7433 - mae: 1.2760 - val_loss: 0.6959 - val_mae: 0.4529
Epoch 3/200
128/128 [==============================] - 33s 257ms/step - loss: 92.8612 - mae: 1.2874 - val_loss: 0.7067 - val_mae: 0.5189
Epoch 4/200
128/128 [==============================] - 33s 256ms/step - loss: 92.0954 - mae: 1.3253 - val_loss: 0.7723 - val_mae: 0.6018
Epoch 5/200
128/128 [==============================] - 32s 253ms/step - loss: 91.1257 - mae: 1.3610 - val_loss: 0.8958 - val_mae: 0.6925
Epoch 6/200
128/128 [==============================] - 33s 254ms/step - loss: 89.5307 - mae: 1.4120 - val_loss: 1.1074 - val_mae: 0.8066
Epoch 7/200
128/128 [==============================] - 34s 262ms/step - loss: 87.3926 - mae: 1.4703 - val_loss: 1.8617 - val_mae: 1.0693
Epoch 8/200
128/128 [====================

[I 2025-07-09 15:18:20,576] Trial 1 finished with value: 0.6959306001663208 and parameters: {'l1': 4.39532161797545e-06, 'l2': 0.0037128161814358298, 'lstm_units_1': 128, 'activation_lstm': 'relu', 'recurrent_dropout_1': 0.3999584193540931, 'dropout_rate': 0.29257334891140624, 'lstm_units_2': 16, 'activation_lstm_2': 'relu', 'recurrent_dropout_2': 0.41689762903071614, 'dense_units': 8, 'activation_dense': 'relu', 'optimizer': 'rmsprop', 'learning_rate': 0.00010189611547405376}. Best is trial 1 with value: 0.6959306001663208.


...

Epoch 1/200
128/128 [==============================] - 38s 258ms/step - loss: 92.5445 - mae: 1.2955 - val_loss: 0.8828 - val_mae: 0.8164
Epoch 2/200
128/128 [==============================] - 33s 255ms/step - loss: 86.4602 - mae: 1.5649 - val_loss: 2.7317 - val_mae: 0.8781
Epoch 3/200
128/128 [==============================] - 32s 253ms/step - loss: 80.4272 - mae: 1.5922 - val_loss: 3.0745 - val_mae: 0.9182
Epoch 4/200
128/128 [==============================] - 32s 252ms/step - loss: 77.1713 - mae: 1.5439 - val_loss: 2.9732 - val_mae: 1.0670
Epoch 5/200
128/128 [==============================] - 32s 253ms/step - loss: 77.3206 - mae: 1.5398 - val_loss: 3.2960 - val_mae: 0.9576
Epoch 6/200
128/128 [==============================] - 33s 254ms/step - loss: 75.3191 - mae: 1.5721 - val_loss: 9.7865 - val_mae: 2.1317
Epoch 7/200
128/128 [==============================] - 32s 251ms/step - loss: 73.6526 - mae: 1.5450 - val_loss: 3.2611 - val_mae: 1.0591
Epoch 8/200
128/128 [====================

[I 2025-07-09 21:01:59,936] Trial 48 finished with value: 0.8828231692314148 and parameters: {'l1': 7.429795019302844e-08, 'l2': 6.328302656544882e-08, 'lstm_units_1': 64, 'activation_lstm': 'tanh', 'recurrent_dropout_1': 0.06186202093598284, 'dropout_rate': 0.1173583684840945, 'lstm_units_2': 32, 'activation_lstm_2': 'relu', 'recurrent_dropout_2': 0.0010631619149471527, 'dense_units': 8, 'activation_dense': 'relu', 'optimizer': 'adam', 'learning_rate': 0.00034508596315262685}. Best is trial 17 with value: 0.34048137068748474.


Epoch 1/200
128/128 [==============================] - 39s 264ms/step - loss: 92.9441 - mae: 1.2717 - val_loss: 0.4442 - val_mae: 0.5234
Epoch 2/200
128/128 [==============================] - 32s 248ms/step - loss: 92.5401 - mae: 1.3378 - val_loss: 0.6201 - val_mae: 0.6665
Epoch 3/200
128/128 [==============================] - 33s 255ms/step - loss: 92.1556 - mae: 1.3987 - val_loss: 0.7382 - val_mae: 0.7487
Epoch 4/200
128/128 [==============================] - 32s 252ms/step - loss: 91.6641 - mae: 1.4389 - val_loss: 1.0509 - val_mae: 0.8921
Epoch 5/200
128/128 [==============================] - 33s 257ms/step - loss: 91.0872 - mae: 1.5044 - val_loss: 1.1618 - val_mae: 0.8827
Epoch 6/200
128/128 [==============================] - 32s 251ms/step - loss: 90.4512 - mae: 1.5256 - val_loss: 1.2383 - val_mae: 0.8686
Epoch 7/200
128/128 [==============================] - 32s 252ms/step - loss: 89.9582 - mae: 1.4929 - val_loss: 1.5588 - val_mae: 0.9449
Epoch 8/200
128/128 [====================

[I 2025-07-09 21:08:04,051] Trial 49 finished with value: 0.4442257881164551 and parameters: {'l1': 2.0725827535071402e-08, 'l2': 1.9492876393439258e-08, 'lstm_units_1': 32, 'activation_lstm': 'tanh', 'recurrent_dropout_1': 0.08225426905252003, 'dropout_rate': 0.15635727862533583, 'lstm_units_2': 16, 'activation_lstm_2': 'tanh', 'recurrent_dropout_2': 0.08139926240643466, 'dense_units': 16, 'activation_dense': 'relu', 'optimizer': 'adam', 'learning_rate': 0.00012708753581021367}. Best is trial 17 with value: 0.34048137068748474.


Mejores parámetros:
{'l1': 3.3692472030315245e-07, 'l2': 8.7876011262569e-08, 'lstm_units_1': 32, 'activation_lstm': 'tanh', 'recurrent_dropout_1': 0.26612587287797906, 'dropout_rate': 0.18703491200890998, 'lstm_units_2': 32, 'activation_lstm_2': 'relu', 'recurrent_dropout_2': 0.10895054417461136, 'dense_units': 8, 'activation_dense': 'relu', 'optimizer': 'adam', 'learning_rate': 0.0005130986400377099}


In [4]:
# Ruta a la base de datos
DB_PATH = "optuna_storage/bbdd.db"

# Cargar el estudio almacenado
study = optuna.load_study(
    study_name=study_name,  # Usar None si solo hay un estudio en la DB, o especificar el nombre
    storage=f"sqlite:///{DB_PATH}"
)

# Obtener los mejores hiperparámetros
best_params = study.best_params
best_value = study.best_value


print("Mejores hiperparámetros encontrados:")
print(best_params)

Mejores hiperparámetros encontrados:
{'l1': 3.3692472030315245e-07, 'l2': 8.7876011262569e-08, 'lstm_units_1': 32, 'activation_lstm': 'tanh', 'recurrent_dropout_1': 0.26612587287797906, 'dropout_rate': 0.18703491200890998, 'lstm_units_2': 32, 'activation_lstm_2': 'relu', 'recurrent_dropout_2': 0.10895054417461136, 'dense_units': 8, 'activation_dense': 'relu', 'optimizer': 'adam', 'learning_rate': 0.0005130986400377099}


In [5]:
l1 = best_params["l1"]
l2 = best_params["l2"]
reg = regularizers.l1_l2(l1=l1, l2=l2)

model = Sequential()
model.add(LSTM(
    units=best_params["lstm_units_1"],
    activation=best_params["activation_lstm"],
    input_shape=(14, 1),
    return_sequences=True,
    recurrent_dropout=best_params["recurrent_dropout_1"],
    kernel_regularizer=reg
))
model.add(Dropout(best_params["dropout_rate"]))

model.add(LSTM(
    units=best_params["lstm_units_2"],
    activation=best_params["activation_lstm_2"],
    recurrent_dropout=best_params["recurrent_dropout_2"],
    kernel_regularizer=reg
))

model.add(Dense(
    units=best_params["dense_units"],
    activation=best_params["activation_dense"],
    kernel_regularizer=reg
))

model.add(Dense(1))

optimizer_name = best_params["optimizer"]
learning_rate = best_params["learning_rate"]

if optimizer_name == "adam":
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
elif optimizer_name == "rmsprop":
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
else:
    optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

model.compile(optimizer=optimizer, loss="mse", metrics=["mae"])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 14, 32)            4352      
                                                                 
 dropout (Dropout)           (None, 14, 32)            0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 8)                 264       
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 12,945
Trainable params: 12,945
Non-trainable params: 0
_________________________________________________________________


In [6]:


#early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=200,
    batch_size=128,
    verbose=1,
    #callbacks=[early_stopping]
)
val_loss = min(history.history["val_loss"])
print(f"Mejor validación MSE: {val_loss:.4f}")

Epoch 1/200
128/128 [==============================] - 41s 250ms/step - loss: 92.4079 - mae: 1.3757 - val_loss: 1.0713 - val_mae: 0.9333
Epoch 2/200
128/128 [==============================] - 31s 246ms/step - loss: 87.6363 - mae: 1.6160 - val_loss: 1.4445 - val_mae: 0.9722
Epoch 3/200
128/128 [==============================] - 30s 238ms/step - loss: 82.2088 - mae: 1.6940 - val_loss: 1.4131 - val_mae: 0.9376
Epoch 4/200
128/128 [==============================] - 31s 238ms/step - loss: 79.3094 - mae: 1.6312 - val_loss: 6.1503 - val_mae: 1.6223
Epoch 5/200
128/128 [==============================] - 30s 237ms/step - loss: 75.5787 - mae: 1.6375 - val_loss: 2.1370 - val_mae: 0.9408
Epoch 6/200
128/128 [==============================] - 30s 237ms/step - loss: 74.2910 - mae: 1.5644 - val_loss: 5.7674 - val_mae: 1.4673
Epoch 7/200
128/128 [==============================] - 30s 236ms/step - loss: 80.9130 - mae: 1.3536 - val_loss: 0.4942 - val_mae: 0.4815
Epoch 8/200
128/128 [====================

In [7]:
def predecir_t2(product_id, ultimos_12_meses,mes, quarter):
    """
    product_id: ID del producto (ej: 'P001').
    ultimos_12_meses: Array con las tn de los últimos 12 meses (en orden cronológico).
    """
    scaler = scalers[product_id]
    ultimos_12_scaled = scaler.transform(np.array(ultimos_12_meses).reshape(-1, 1))
    mes_scaled = (mes - 1) / 11  # Normalizado a [0, 1]
    quarter_scaled = (quarter - 1) / 3  # quarter ∈ [0, 3]
    ultimos_12_scaled = np.concatenate([
            ultimos_12_scaled.reshape(-1, 1),
            np.array([[mes_scaled], [quarter_scaled]])
        ], axis=0)
    X_new = ultimos_12_scaled.reshape(1, 14, 1)
    y_pred_scaled = model.predict(X_new)
    return (y_pred_scaled * scaler.scale_) + scaler.center_

# Ejemplo: Predecir para el primer producto en el test set
producto_ejemplo = 20001
ultimos_12_ejemplo = df_pivot.loc[producto_ejemplo].iloc[-12:].values
prediccion = predecir_t2(producto_ejemplo, ultimos_12_ejemplo)
print(f"Predicción t+2 para {producto_ejemplo}: {prediccion[0][0]:.2f} tn")

TypeError: predecir_t2() missing 2 required positional arguments: 'mes' and 'quarter'

In [8]:
from tensorflow.keras.models import load_model
os.makedirs("modelos", exist_ok=True)
# Guardar el modelo (formato HDF5)
model.save(f'modelos/{study_name}.h5')


In [9]:
# Guardar pesos
model.save_weights(f'modelos/pesos_{study_name}.h5')


In [10]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
productos_ok = productos_ok["product_id"].unique()
def predecir_todos(productos_a_predecir):
    predicciones = {}
    for producto in productos_a_predecir:
        ultimos_12 = df_pivot.loc[producto].iloc[-12:].values
        mes = pd.to_datetime(df_pivot.columns[-1], format="%Y%m").month
        quarter = pd.to_datetime(df_pivot.columns[-1], format="%Y%m").quarter
        prediccion = predecir_t2(producto, ultimos_12,mes, quarter)
        predicciones[producto] = prediccion[0][0]
    return predicciones
predicciones = predecir_todos(productos_ok)
predicciones_df = pd.DataFrame.from_dict(predicciones, orient='index', columns=['prediccion_t2'])
predicciones_df.index.name = 'product_id'
predicciones_df.reset_index(inplace=True)
predicciones_df

1/1 [==============================] - 0s 43ms/step


,product_id,prediccion_t2
0,20001,1361.287519
1,20002,1043.858051
2,20003,788.943969
3,20004,548.201077
4,20005,459.580126
...,...,...
775,21263,0.009634
776,21265,0.016413
777,21266,0.017833
778,21267,0.014523


In [11]:
predicciones_df.columns = ["product_id", "tn"]
predicciones_df.to_csv(f"../../results/{study_name}.csv", index=False, sep=",")